In [69]:
# You can make a copy, then change and run the code
# path need to be changed
# ~2 hours to train 
import sys 
sys.path.append('/content/gdrive/My Drive/final_pj')
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
## some config values 
embed_size = 300 # how big is each word vector
max_features = 50000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 70 # max number of words in a question to use

In [0]:
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from tensorflow.keras.layers import LSTM, GRU
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from tensorflow.keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import *


from tensorflow.keras import preprocessing
import gensim

In [0]:
def is_number(text):
  try:
    if text[-1] == 's':
      text = text[:-1]
  except:
    pass
  
  try:
    if text[-1] == 'h' and text[-2] == 't':
      text = text[:-2]
  except:
    pass
    
  for digits in text:
    if digits not in ['0','1','2','3','4','5','6','7','8','9']:
      return False
  return True

In [0]:
def load_and_prec():
    train_df = pd.read_csv("/content/gdrive/My Drive/final_pj/train.csv")
    test_df = pd.read_csv("/content/gdrive/My Drive/final_pj/test.csv")
    print("Train shape : ",train_df.shape)
    print("Test shape : ",test_df.shape)
    
    ## fill up the missing values
    train_X = train_df["question_text"].fillna("_##_").values
    test_X = test_df["question_text"].fillna("_##_").values

    ## Tokenize the sentences
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(train_X))
    train_X = tokenizer.texts_to_sequences(train_X)
    test_X = tokenizer.texts_to_sequences(test_X)

    ## Pad the sentences 
    train_X = pad_sequences(train_X, maxlen=maxlen)
    test_X = pad_sequences(test_X, maxlen=maxlen)

    ## Get the target values
    train_y = train_df['target'].values
    
    #shuffling the data
    np.random.seed(2018)
    trn_idx = np.random.permutation(len(train_X))

    train_X = train_X[trn_idx]
    train_y = train_y[trn_idx]
    
    return train_X, test_X, train_y, tokenizer.word_index

In [0]:
def load_and_prec2(stopwords):
    train_df = pd.read_csv("/content/gdrive/My Drive/final_pj/train.csv")
    test_df = pd.read_csv("/content/gdrive/My Drive/final_pj/test.csv")
    print("Train shape : ",train_df.shape)
    print("Test shape : ",test_df.shape)
    
    ## fill up the missing values
    train_X_raw = train_df["question_text"].fillna("").values
    train_X_raw = [preprocessing.text.text_to_word_sequence(text, lower=False) for text in train_X_raw]
    train_X = []
    for line in train_X_raw:
      tmp = []
      for item in line:
        if item in stopwords:
          continue
        if is_number(item):
          tmp.append('NUM')
        else:
          tmp.append(item)
      train_X.append(' '.join(tmp))
    del train_X_raw
    
    test_X_raw = test_df["question_text"].fillna("").values
    test_X_raw = [preprocessing.text.text_to_word_sequence(text, lower=False) for text in test_X_raw]
    test_X = []
    for line in test_X_raw:
      tmp = []
      for item in line:
        if item in stopwords:
          continue
        if is_number(item):
          tmp.append('NUM')
        else:
          tmp.append(item)
      test_X.append(' '.join(tmp))
    del test_X_raw
  
    ## Tokenize the sentences
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(train_X))
    train_X = tokenizer.texts_to_sequences(train_X)
    test_X = tokenizer.texts_to_sequences(test_X)

    ## Pad the sentences 
    train_X = pad_sequences(train_X, maxlen=maxlen, value=0)
    test_X = pad_sequences(test_X, maxlen=maxlen, value=0)

    ## Get the target values
    train_y = train_df['target'].values
    
    #shuffling the data
    np.random.seed(2018)
    trn_idx = np.random.permutation(len(train_X))

    train_X = train_X[trn_idx]
    train_y = train_y[trn_idx]
    
    return train_X, test_X, train_y, tokenizer.word_index

In [0]:
def load_glove(word_index):
    EMBEDDING_FILE = '/content/gdrive/My Drive/final_pj/embeddings/glove.840B.300d.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
            
    return embedding_matrix 
    
def load_fasttext(word_index):    
    EMBEDDING_FILE = '/content/gdrive/My Drive/Final Project/4995 DL/basic classification/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector

    return embedding_matrix

def load_para(word_index):
    EMBEDDING_FILE = '/content/gdrive/My Drive/Final Project/4995 DL/basic classification/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    
    return embedding_matrix


In [0]:
def load_fasttext2(word_index): 
  WIKI_NEWS = '/content/gdrive/My Drive/final_pj/embeddings/wiki-news-300d-1M.vec'
  wikinews_model = gensim.models.KeyedVectors.load_word2vec_format(WIKI_NEWS, binary=False)
  
  nb_words = min(max_features, len(word_index))
  embedding_matrix = np.zeros((nb_words, embed_size))
  for words, i in word_index.items():
    if i >= max_features: 
      continue
    if words in wikinews_model:
      embedding_matrix[i] = wikinews_model[words]
    else:
      embedding_matrix[i] = wikinews_model['UNK']
  
  # embedding_matrix[max_features] = wikinews_model['UNK']
  embedding_matrix[0] = np.array([0.0]*embed_size)
  del wikinews_model
  return embedding_matrix

def load_glove2(word_index):
  EMBEDDING_FILE = '/content/gdrive/My Drive/final_pj/embeddings/glove.840B.300d.txt'
  def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
  embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

  all_embs = np.stack(embeddings_index.values())
  emb_mean,emb_std = all_embs.mean(), all_embs.std()
  embed_size = all_embs.shape[1]

  # word_index = tokenizer.word_index
  nb_words = min(max_features, len(word_index))
  embedding_matrix = np.zeros((nb_words, embed_size))
  for word, i in word_index.items():
    if i >= max_features: 
      continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
      embedding_matrix[i] = embedding_vector
    else:
      embedding_matrix[i] = embeddings_index.get('UNK')
      
  # embedding_matrix[max_features] = embeddings_index.get('UNK')
  embedding_matrix[0] = np.array([0.0]*embed_size)
  del embeddings_index
  return embedding_matrix

def load_googlenews(word_index): 
  GoogleNews = '/content/gdrive/My Drive/final_pj/embeddings/GoogleNews-vectors-negative300.bin.gz'
  googlenews_model = gensim.models.KeyedVectors.load_word2vec_format(GoogleNews, binary=True)
  
  nb_words = min(max_features, len(word_index))
  embedding_matrix = np.zeros((nb_words, embed_size))
  print(nb_words)
  for words, i in word_index.items():
    if i >= max_features: 
      continue
    if words in googlenews_model:
      embedding_matrix[i] = googlenews_model[words]
    else:
      embedding_matrix[i] = googlenews_model['UNK']
  
  # embedding_matrix[max_features] = googlenews_model['UNK']
  embedding_matrix[0] = np.array([0.0]*embed_size)
  del googlenews_model
  return embedding_matrix


In [0]:
# https://www.kaggle.com/hireme/fun-api-keras-f1-metric-cyclical-learning-rate/code

class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())
    

def f1(y_true, y_pred):
    '''
    metric from here 
    https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
    '''
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
# https://www.kaggle.com/strideradu/word2vec-and-gensim-go-go-go
def train_pred(model, train_X, train_y, val_X, val_y, epochs=2, callback=None):
    for e in range(epochs):
        model.fit(train_X, train_y, batch_size=512, epochs=1, validation_data=(val_X, val_y), callbacks = callback, verbose=0)
        pred_val_y = model.predict([val_X], batch_size=1024, verbose=0)

        best_score = metrics.f1_score(val_y, (pred_val_y > 0.33).astype(int))
        print("Epoch: ", e, "-    Val F1 Score: {:.4f}".format(best_score))

    pred_test_y = model.predict([test_X], batch_size=1024, verbose=0)
    print('=' * 60)
    return pred_val_y, pred_test_y, best_score

In [0]:
# new implementation
infile = open("/content/gdrive/My Drive/final_pj/embeddings/stopwords.txt")
stopwords = [line[:-1] for line in infile.readlines()]

In [79]:
# new implementation
train_X, test_X, train_y, word_index = load_and_prec2(stopwords)


Train shape :  (1306122, 3)
Test shape :  (375806, 2)


In [80]:
embedding_matrix = load_glove2(word_index)
np.shape(embedding_matrix)

(50000, 300)

In [33]:
# embedding_matrix = load_googlenews(word_index)

50000


In [0]:
# https://www.kaggle.com/ryanzhang/tfidf-naivebayes-logreg-baseline

def threshold_search(y_true, y_proba):
    best_threshold = 0
    best_score = 0
    for threshold in [i * 0.01 for i in range(100)]:
        score = f1_score(y_true=y_true, y_pred=y_proba > threshold)
        if score > best_score:
            best_threshold = threshold
            best_score = score
    search_result = {'threshold': best_threshold, 'f1': best_score}
    return search_result

In [0]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,CuDNNLSTM,LSTM,Dropout

In [0]:
def model_lstm_atten(embedding_matrix_1):
    
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix_1], trainable=False)(inp)
    x = SpatialDropout1D(0.1)(x)
    x = Bidirectional(CuDNNLSTM(40, return_sequences=True))(x)
    y = Bidirectional(CuDNNGRU(40, return_sequences=True))(x)
    
    # atten_1 = Attention(maxlen)(x) # skip connect
    # atten_2 = Attention(maxlen)(y)
    avg_pool = GlobalAveragePooling1D()(y)
    max_pool = GlobalMaxPooling1D()(y)
    
    # conc = concatenate([atten_1, atten_2, avg_pool, max_pool])
    
    # conc = Dense(16, activation="relu")(conc)
    y = Flatten()(y)
    conc = Dense(16, activation="relu")(y)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1])
    print(model.summary())
    return model

In [0]:
DATA_SPLIT_SEED = 2018
clr = CyclicLR(base_lr=0.001, max_lr=0.002,
               step_size=300., mode='exp_range',
               gamma=0.99994)

train_meta = np.zeros(train_y.shape)
test_meta = np.zeros(test_X.shape[0])
splits = list(StratifiedKFold(n_splits=4, shuffle=True, random_state=DATA_SPLIT_SEED).split(train_X, train_y))
for idx, (train_idx, valid_idx) in enumerate(splits):
        X_train = train_X[train_idx]
        y_train = train_y[train_idx]
        X_val = train_X[valid_idx]
        y_val = train_y[valid_idx]
        model = model_lstm_atten(embedding_matrix)
        pred_val_y, pred_test_y, best_score = train_pred(model, X_train, y_train, X_val, y_val, epochs = 8, callback = [clr,])
        train_meta[valid_idx] = pred_val_y.reshape(-1)
        test_meta += pred_test_y.reshape(-1) / len(splits)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 70)                0         
_________________________________________________________________
embedding_14 (Embedding)     (None, 70, 300)           15000000  
_________________________________________________________________
spatial_dropout1d_13 (Spatia (None, 70, 300)           0         
_________________________________________________________________
bidirectional_24 (Bidirectio (None, 70, 80)            109440    
_________________________________________________________________
bidirectional_25 (Bidirectio (None, 70, 80)            29280     
_________________________________________________________________
flatten_5 (Flatten)          (None, 5600)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 16)                89616     
__________

In [0]:
sub = pd.read_csv('/content/gdrive/My Drive/Final Project/4995 DL/basic classification/sample_submission.csv')
sub.prediction = test_meta > 0.33
sub.to_csv("/content/gdrive/My Drive/Final Project/4995 DL/basic classification/submission.csv", index=False)

In [ ]:
# counting unknown symbols from the embedding

In [0]:
from keras import preprocessing
import pandas as pd
import gensim

# count the number of unknown words
train_df = pd.read_csv("/content/gdrive/My Drive/final_pj/train.csv")
    
## fill up the missing values
train_X = train_df["question_text"]
train_X = [preprocessing.text.text_to_word_sequence(text, lower=False) for text in train_X]
print(train_X[0])

['How', 'did', 'Quebec', 'nationalists', 'see', 'their', 'province', 'as', 'a', 'nation', 'in', 'the', '1960s']


In [0]:
GoogleNews = '/content/gdrive/My Drive/final_pj/embeddings/GoogleNews-vectors-negative300.bin.gz'
googlenews_model = gensim.models.KeyedVectors.load_word2vec_format(GoogleNews, binary=True)
word_list = googlenews_model.wv.vocab

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


In [0]:
total_words = 0
total_sent = 0
unk_sent = 0
unk_words = 0
for line in train_X:
  total_sent += 1
  
  flag = False
  for w in line:
    total_words += 1
    if w not in word_list:
      if not is_number(w) and w not in stopwords:
        unk_words += 1
        if flag == False:
          flag = True
          unk_sent += 1
print('total_sentence:', total_sent)
print('sentence with UNK:', unk_sent)
print('total_words:', total_words)
print('words with UNK:', unk_words)

total_sentence: 1306122
sentence with UNK: 354582
total_words: 16856728
words with UNK: 474521


In [0]:
WIKI_NEWS = '/content/gdrive/My Drive/final_pj/embeddings/wiki-news-300d-1M.vec'
wikinews_model = gensim.models.KeyedVectors.load_word2vec_format(WIKI_NEWS, binary=False)
word_list = wikinews_model.wv.vocab

False
True


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
total_words = 0
total_sent = 0
unk_sent = 0
unk_words = 0
for line in train_X:
  total_sent += 1
  
  flag = False
  for w in line:
    total_words += 1
    if w not in word_list:
      if not is_number(w) and w not in stopwords:
        unk_words += 1
        if flag == False:
          flag = True
          unk_sent += 1
print('total_sentence:', total_sent)
print('sentence with UNK:', unk_sent)
print('total_words:', total_words)
print('words with UNK:', unk_words)

total_sentence: 1306122
sentence with UNK: 319679
total_words: 16856728
words with UNK: 409323


In [0]:
GLOVE_NEWS = '/content/gdrive/My Drive/final_pj/embeddings/glove.840B.300d.txt'
glove_news = open(GLOVE_NEWS, 'rb')
word_set = set()
for line in glove_news:
  line = line.decode(encoding="utf8", errors='ignore').rstrip('\n')
  word_set.add(line.split()[0])
glove_news.close()
print('NUM' in word_set)
print('UNK' in word_set)
print('EOS' in word_set)
print('' in word_set)

True
True
True
False


In [0]:
total_words = 0
total_sent = 0
unk_sent = 0
unk_words = 0
for line in train_X:
  total_sent += 1
  
  flag = False
  for w in line:
    total_words += 1
    if w not in word_set:
      if not is_number(w) and w not in stopwords:
        unk_words += 1
        if flag == False:
          flag = True
          unk_sent += 1
print('total_sentence:', total_sent)
print('sentence with UNK:', unk_sent)
print('total_words:', total_words)
print('words with UNK:', unk_words)

total_sentence: 1306122
sentence with UNK: 173013
total_words: 16856728
words with UNK: 208634


In [0]:
PARA_NEWS = '/content/gdrive/My Drive/final_pj/embeddings/paragram_300_sl999.txt'
para_news = open(PARA_NEWS, 'rb')
word_set = set()
for line in para_news:
  line = line.decode(encoding="utf8", errors='ignore').rstrip('\n')
  word_set.add(line.split()[0])

para_news.close()

print('NUM' in word_set)
print('UNK' in word_set)

False
False


In [0]:
total_words = 0
total_sent = 0
unk_sent = 0
unk_words = 0
for line in train_X:
  total_sent += 1
  
  flag = False
  for w in line:
    total_words += 1
    if w not in word_set:
      if not is_number(w) and w not in stopwords:
        unk_words += 1
        if flag == False:
          flag = True
          unk_sent += 1
print('total_sentence:', total_sent)
print('sentence with UNK:', unk_sent)
print('total_words:', total_words)
print('words with UNK:', unk_words)

total_sentence: 1306122
sentence with UNK: 1302879
total_words: 16856728
words with UNK: 2925797
